In [1]:
import pandas as pd
import numpy as np

In [2]:
# Define the input year and next year
year = 2024  # Current year for the main data
next_year = year + 1  # Next year for the reference data

# Load the surf spot data with monthly mean values (VV, VH) for the current year
monthly_data = pd.read_csv(f"SurfSpots_Sentinel(Monthly Mean {year}).csv")

# Load the reference CSV for the next year's values (VV, VH)
reference_data = pd.read_csv(f"SurfSpots_Sentinel(Monthly Mean {year}).csv")

# Ensure the 'Date' columns are in datetime format
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])
reference_data['Date'] = pd.to_datetime(reference_data['Date'])

# Create a new column for daily date range (full year)
daily_date_range = pd.date_range(start=f"{year}-01-01", end=f"{year}-12-31", freq='D')

# Create a new DataFrame to store daily values
daily_results = []

In [3]:
# Loop through each surf spot
for _, surf_spot in monthly_data.groupby('Surf Spot'):
    for surf_spot_name, group in surf_spot.groupby('Surf Spot'):
        # Sort by date (ensure it's in order)
        group = group.sort_values('Date')

        # Convert date to ordinal (days since epoch)
        group['Date_ordinal'] = group['Date'].apply(lambda x: x.toordinal())
        daily_date_range_ordinal = daily_date_range.to_series().apply(lambda x: x.toordinal())

        # Interpolation for VV and VH using linear interpolation (except December)
        vv_interpolated = np.interp(daily_date_range_ordinal, group['Date_ordinal'], group['VV'])
        vh_interpolated = np.interp(daily_date_range_ordinal, group['Date_ordinal'], group['VH'])

        # Handle December by using January of next year as reference
        december_index = daily_date_range.month == 12

        # Get the January values from the next year's reference data
        january_next_year_data = reference_data[reference_data['Surf Spot'] == surf_spot_name]
        january_next_year_vv = january_next_year_data[january_next_year_data['Date'].dt.month == 1]['VV'].values[0]
        january_next_year_vh = january_next_year_data[january_next_year_data['Date'].dt.month == 1]['VH'].values[0]

        # Apply the January next year values to December of the current year
        vv_interpolated[december_index] = january_next_year_vv
        vh_interpolated[december_index] = january_next_year_vh

        # Append the daily values for this surf spot
        for i, date in enumerate(daily_date_range):
            daily_results.append({
                'Date': date,
                'Surf Spot': surf_spot_name,
                'VV': vv_interpolated[i],
                'VH': vh_interpolated[i]
            })

# Convert to DataFrame
daily_df = pd.DataFrame(daily_results)

In [4]:
# Save the result to a new CSV file
daily_df.to_csv(f"Surf_Spots_VV_VH_Daily_{year}_with_December_filled.csv", index=False)

print(f"Interpolation complete. Daily values with December filled saved to 'Surf_Spots_VV_VH_Daily_{year}_with_December_filled.csv'")

Interpolation complete. Daily values with December filled saved to 'Surf_Spots_VV_VH_Daily_2024_with_December_filled.csv'
